In [31]:
# Load libraries
import pandas as pd
import numpy as np
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from pymongo import MongoClient
import datetime

In [32]:
# Load dataset
path = "D:/OE0t11824/ATT/mongo_dump/vijay/SOTrainingData.csv"
names = ['cpu', 'memory', 'network', 'diskSpace', 'numOfUser','noOfConn', 'responseTime','flaps', 'result','VNFSource']
rawdata = pd.read_csv(path, names=names)

client = MongoClient(host='localhost:27017')
db = client.dcomp
#rawdata_collection = db.rawdata
predictdata_collection = db.predictdata_collection
datacount_collection = db.datacount_collection

#insert/update dates to raw data
dates=[]
lag =0
for row in rawdata['result']: 
    dates.append(datetime.datetime.now() + datetime.timedelta(0,10+lag))
    lag = lag + 50         
rawdata['date'] = dates



In [33]:

# Split-out validation dataset
array = rawdata.values
X = array[:,0:8]
Y = array[:,8]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)


In [34]:
rawdata["id"] = rawdata.index + 1
# make prediction (KNN)
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, Y_train)
knn_predictions = knn_model.predict(X_validation)
print(accuracy_score(Y_validation,knn_predictions))


0.909090909091


In [35]:

# Load actual production VNF dataset which has unknown results
path = "D:/OE0t11824/ATT/mongo_dump/vijay/SOVNFData.csv"
names = ['cpu', 'memory', 'network', 'diskSpace', 'numOfUser','noOfConn', 'responseTime','flaps', "result",'VNFSource']
testdataset = pd.read_csv(path, names=names)


In [36]:
# Split-out validation dataset
testarray = testdataset.values
testX = testarray[:,0:8]
testY = testarray[:,8]
testS = testarray[:,9]


In [37]:
# make prediction (KNN)
testknn_predictions = knn_model.predict(testX)
#print(testknn_predictions)
df_data = pd.DataFrame(testX,columns=['cpu', 'memory', 'network', 'diskSpace', 'numOfUser','noOfConn', 'responseTime','flaps'])
df_result = pd.DataFrame(testY,columns=['result'])
df_vnfsoruce = pd.DataFrame(testS,columns=['VNFSource'])
df_prediction = pd.DataFrame(testknn_predictions,columns=['prediction'])

predictdata = pd.concat([df_data, df_result,df_vnfsoruce, df_prediction], axis=1)

#print(predictdata)
#print(predictdata['prediction'])

In [38]:
predictdata['date'] = rawdata['date']
  
print(len(predictdata))
print(len(predictdata[predictdata['prediction'] == 'Disk Full']))


      

110
15


In [39]:
predictdata['date'] = rawdata['date']

# count based on Prediction status


passcount = len(predictdata[predictdata['prediction'] == 'Pass'])
sfcount = len(predictdata[predictdata['prediction'] == 'Service Failure'])
cpumemorycount = len(predictdata[predictdata['prediction'] == 'CPU Memory'])
memoryleakcount = len(predictdata[predictdata['prediction'] == 'Memory Leak'])
responsecount = len(predictdata[predictdata['prediction'] == 'Response'])
memorycount = len(predictdata[predictdata['prediction'] == 'High Memory'])
connectioncount = len(predictdata[predictdata['prediction'] == 'Connections'])
diskfullcount = len(predictdata[predictdata['prediction'] == 'Disk Full'])
slowcount=cpumemorycount + responsecount+memorycount+connectioncount
failcount = sfcount+memoryleakcount+diskfullcount
#print(predictdata)
print(passcount)
print(memoryleakcount)
print(sfcount)
print(cpumemorycount)
print(responsecount)
print(memorycount)
print(connectioncount)
print(diskfullcount)
print(slowcount)
print(failcount)
#accuracy = accuracy *100

45
10
0
17
6
14
3
15
40
25


In [40]:
cnt_dict = {}
cnt_dict['pass']= passcount
cnt_dict['serviceFaliure']=sfcount
cnt_dict['cpuMemory']=cpumemorycount
cnt_dict['memoryLeak']=memoryleakcount
cnt_dict['response']=responsecount
cnt_dict['highMemory']=memorycount
cnt_dict['connections']=connectioncount
cnt_dict['diskFull']=diskfullcount
cnt_dict['slow']=slowcount
cnt_dict['fail']=failcount
#cnt_dict['accuracy']=accuracy

In [41]:
columns=list(cnt_dict.keys())
values = list(cnt_dict.values())
arr_len = len(values)
df_cnt=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
df_cnt['id'] = df_cnt.index + 1
#print(rawdata)
#print(predictdata)
#print(df_cnt)

In [42]:
#db.rawdata_collection.insert_many(rawdata.to_dict('records'))
db.predictdata_collection.delete_many({})
db.datacount_collection.delete_many({})
db.predictdata_collection.insert_many(predictdata.to_dict('records'))
db.datacount_collection.insert_many(df_cnt.to_dict('records'))